In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ======================== CELL 1: DEPENDENCIES ==========================
import subprocess
import sys

print('🔧 Installing dependencies...')
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "pyarrow"], capture_output=True, check=False)
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "pyarrow==15.0.2", "keybert", "rank-bm25", "faiss-cpu", "sacremoses"], check=True)
print('✅ Dependencies installed. RESTART kernel and run Cell 2.')


In [2]:
# ===================== CELL 2: IMPORTS & SETUP (SYMPTOMS TRIAGE) =====================
import warnings
warnings.filterwarnings("ignore")

import re
import json
import time
import pickle
import os
from pathlib import Path
from dataclasses import dataclass
from typing import List, Dict, Tuple

import numpy as np
import torch
import faiss
from sentence_transformers import SentenceTransformer
from nltk.tokenize import word_tokenize, sent_tokenize
from rank_bm25 import BM25Okapi

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 Using device: {device}")

@dataclass
class DomainConfig:
    name: str = "symptoms_triage"
    dataset_path: str = "dataset_file = /kaggle/input/medquad-medical-question-answer-for-ai-research/medquad-train.json"  # adjust if needed
    chunk_window: int = 3
    chunk_stride: int = 1
    embed_model: str = "sentence-transformers/all-MiniLM-L6-v2"

config = DomainConfig()

print(f"📋 Building index for: {config.name}")
print(f"📦 Dataset path: {config.dataset_path}")


🔧 Using device: cuda
📋 Building index for: symptoms_triage
📦 Dataset path: dataset_file = /kaggle/input/medquad-medical-question-answer-for-ai-research/medquad-train.json


In [3]:
# ===================== CELL 3: DATASET LOADING FROM KAGGLE =====================
import glob

# Direct path to load JSON dataset
dataset_dir = '/kaggle/input/medquad-medical-question-answer-for-ai-research/'

# First check what files exist
print(f"\n📍 Checking directory: {dataset_dir}")
all_files = os.listdir(dataset_dir)
print(f"Files found: {all_files}")

# Look for JSON files first
json_pattern = os.path.join(dataset_dir, '*.json')
json_files = glob.glob(json_pattern)

if json_files:
    dataset_file = json_files[0]
    print(f"\n📥 Loading JSON file: {dataset_file}...")
    with open(dataset_file, 'r', encoding='utf-8') as f:
        dataset = json.load(f)
    print(f"✅ Loaded dataset with {len(dataset)} entries")
else:
    # Try to find and load medquad-train.json specifically
    dataset_file = os.path.join(dataset_dir, 'medquad-train.json')
    if os.path.exists(dataset_file):
        print(f"\n📥 Loading {dataset_file}...")
        with open(dataset_file, 'r', encoding='utf-8') as f:
            dataset = json.load(f)
        print(f"✅ Loaded dataset with {len(dataset)} entries")
    else:
        # Load from CSV if JSON doesn't exist
        csv_file = os.path.join(dataset_dir, 'medquad.csv')
        if os.path.exists(csv_file):
            print(f"\n⚠️ No JSON file found. Loading CSV: {csv_file}...")
            import pandas as pd
            df = pd.read_csv(csv_file)
            dataset = df.to_dict('records')
            print(f"✅ Loaded dataset with {len(dataset)} entries from CSV")
            print(f"   Columns: {list(df.columns)}")
        else:
            raise FileNotFoundError(f"❌ No dataset files found in {dataset_dir}")

# Extraction function
def extract_qa_pairs(dataset) -> list:
    qa_data = []
    for idx, row in enumerate(dataset):
        try:
            # Flexible key matching
            question = row.get("question", "") or row.get("Question", "") or row.get("instruction", "")
            answer = row.get("answer", "") or row.get("Answer", "") or row.get("output", "")
            
            if question and answer and len(answer) > 15:
                qa_data.append({
                    "question": str(question).strip(),
                    "answer": str(answer).strip(),
                    "source_id": idx
                })
        except Exception as e:
            if idx < 3:
                print(f"⚠️ Skipped row {idx}: {e}")
    return qa_data

# Extract Q&A pairs
qa_data = extract_qa_pairs(dataset)
print(f"✅ Extracted {len(qa_data)} symptoms/triage Q&A pairs")

if len(qa_data) < 100:
    print(f"⚠️ Only {len(qa_data)} Q&A pairs extracted! Check dataset structure.")
    print(f"Sample row keys: {list(dataset[0].keys()) if dataset else 'empty'}")
else:
    print(f"\n📝 Sample triage Q: {qa_data[0]['question'][:120]}...")
    print(f"   Sample triage A: {qa_data[0]['answer'][:120]}...")


📍 Checking directory: /kaggle/input/medquad-medical-question-answer-for-ai-research/
Files found: ['medquad.csv']

⚠️ No JSON file found. Loading CSV: /kaggle/input/medquad-medical-question-answer-for-ai-research/medquad.csv...
✅ Loaded dataset with 16412 entries from CSV
   Columns: ['question', 'answer', 'source', 'focus_area']
✅ Extracted 16406 symptoms/triage Q&A pairs

📝 Sample triage Q: What is (are) Glaucoma ?...
   Sample triage A: Glaucoma is a group of diseases that can damage the eye's optic nerve and result in vision loss and blindness. While gla...


In [4]:
# ===================== CELL 4: TEXT CHUNKING =====================
def create_chunks(data: List[Dict], window: int = 3, stride: int = 1, min_chars: int = 10) -> List[Dict]:
    chunks = []
    for item in data:
        text = item.get("answer", "")
        if not text or len(text) < min_chars:
            continue
        sentences = sent_tokenize(text)
        if not sentences:
            continue
        if len(sentences) <= window:
            chunks.append({
                "chunk": " ".join(sentences),
                "source_idx": item.get("source_id", -1),
                "chunk_id": len(chunks)
            })
            continue
        for i in range(0, max(1, len(sentences) - window + 1), stride):
            chunks.append({
                "chunk": " ".join(sentences[i:i + window]),
                "source_idx": item.get("source_id", -1),
                "chunk_id": len(chunks)
            })
    return chunks

print(f"🔪 Creating chunks (window={config.chunk_window}, stride={config.chunk_stride}, min_chars=10)...")
chunks = create_chunks(qa_data, window=config.chunk_window, stride=config.chunk_stride, min_chars=10)
print(f"✅ Created {len(chunks)} chunks")

if len(chunks) == 0:
    print("[❌ ERROR] No chunks created! Check QA extraction or chunking settings.")
else:
    print("\n📝 Sample chunks:")
    for i, chunk in enumerate(chunks[:3]):
        print(f"{i+1}. {chunk['chunk'][:120]}...")


🔪 Creating chunks (window=3, stride=1, min_chars=10)...
✅ Created 147907 chunks

📝 Sample chunks:
1. Glaucoma is a group of diseases that can damage the eye's optic nerve and result in vision loss and blindness. While gla...
2. While glaucoma can strike anyone, the risk is much greater for people over 60. How Glaucoma Develops  There are several ...
3. How Glaucoma Develops  There are several different types of glaucoma. Most of these involve the drainage system within t...


In [5]:
# ===================== CELL 5: BUILD FAISS INDEX =====================
if len(chunks) == 0:
    raise ValueError("❌ ERROR: No chunks created! Check previous cell.")

print(f"📦 Loading embedder: {config.embed_model}")
embedder = SentenceTransformer(config.embed_model, device=device)
print(f"✅ Embedder loaded")

id2doc = [chunk["chunk"] for chunk in chunks]
print(f"📊 Encoding {len(id2doc)} chunks...")

if len(id2doc) == 0:
    raise ValueError("❌ ERROR: id2doc is empty! No valid chunks to encode.")

embeddings = embedder.encode(
    id2doc,
    normalize_embeddings=True,
    show_progress_bar=True,
    batch_size=64,
    convert_to_numpy=True
).astype('float32')

print(f"✅ Embeddings shape: {embeddings.shape}")

if len(embeddings.shape) != 2:
    raise ValueError(f"❌ ERROR: Invalid embeddings shape {embeddings.shape}. Expected 2D array.")

dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)

print(f"✅ FAISS index built: {index.ntotal} vectors, dimension {dim}")


📦 Loading embedder: sentence-transformers/all-MiniLM-L6-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Embedder loaded
📊 Encoding 147907 chunks...


Batches:   0%|          | 0/2312 [00:00<?, ?it/s]

✅ Embeddings shape: (147907, 384)
✅ FAISS index built: 147907 vectors, dimension 384


In [6]:
# ===================== CELL 6: BUILD BM25 INDEX =====================
print(f"🔨 Building BM25 index...")
bm25_corpus = [word_tokenize(doc.lower()) for doc in id2doc]
bm25 = BM25Okapi(bm25_corpus)
print(f"✅ BM25 index built")


🔨 Building BM25 index...
✅ BM25 index built


In [7]:
# ===================== CELL 7: SAVE INDEXES =====================
index_file = f"{config.name}_faiss.index"
id2doc_file = f"{config.name}_id2doc.pkl"
metadata_file = f"{config.name}_metadata.json"

print(f"💾 Saving FAISS index to {index_file}...")
faiss.write_index(index, index_file)

print(f"💾 Saving id2doc to {id2doc_file}...")
with open(id2doc_file, "wb") as f:
    pickle.dump(id2doc, f)

metadata = {
    "created_at": time.time(),
    "domain": config.name,
    "dataset_path": config.dataset_path,
    "n_vectors": int(index.ntotal),
    "embedding_dim": dim,
    "chunk_window": config.chunk_window,
    "chunk_stride": config.chunk_stride,
    "embed_model": config.embed_model
}

print(f"💾 Saving metadata to {metadata_file}...")
with open(metadata_file, "w") as f:
    json.dump(metadata, f, indent=2)

print("\n" + "="*80)
print("✅ INDEX BUILDING COMPLETE!")
print("="*80)
print(f"📁 Files created:")
print(f"  - {index_file}")
print(f"  - {id2doc_file}")
print(f"  - {metadata_file}")
print(f"\n📊 Statistics:")
print(f"  - Total vectors: {index.ntotal}")
print(f"  - Embedding dimension: {dim}")
print(f"  - Source Q&A pairs: {len(qa_data)}")
print(f"  - Chunks created: {len(chunks)}")
print("="*80)


💾 Saving FAISS index to symptoms_triage_faiss.index...
💾 Saving id2doc to symptoms_triage_id2doc.pkl...
💾 Saving metadata to symptoms_triage_metadata.json...

✅ INDEX BUILDING COMPLETE!
📁 Files created:
  - symptoms_triage_faiss.index
  - symptoms_triage_id2doc.pkl
  - symptoms_triage_metadata.json

📊 Statistics:
  - Total vectors: 147907
  - Embedding dimension: 384
  - Source Q&A pairs: 16406
  - Chunks created: 147907


In [8]:
# ===================== CELL 8: VERIFY INDEX (OPTIONAL) =====================
def test_retrieval(query: str, top_k: int = 5):
    print(f"\n🔍 Testing query: '{query}'")
    query_emb = embedder.encode([query], normalize_embeddings=True, convert_to_numpy=True).astype('float32')
    D, I = index.search(query_emb, top_k)
    print(f"\n📋 Top {top_k} results:")
    for i, (idx, score) in enumerate(zip(I[0], D[0]), 1):
        print(f"\n{i}. Score: {score:.4f}")
        print(f"   {id2doc[idx][:200]}...")

test_queries = [
    "Patient has fever and cough; what is likely diagnosis?",
    "What symptoms suggest meningitis in a child?",
    "When should I see a doctor for a headache?",
    "What are red flag symptoms for chest pain?",
    "What diagnostic tests are recommended for persistent cough?"
]

print("\n" + "="*80)
print("🧪 TESTING SYMPTOMS / TRIAGE INDEX")
print("="*80)

for query in test_queries:
    test_retrieval(query, top_k=3)



🧪 TESTING SYMPTOMS / TRIAGE INDEX

🔍 Testing query: 'Patient has fever and cough; what is likely diagnosis?'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📋 Top 3 results:

1. Score: 0.7012
   -  Fever for no known reason or frequent infections. - Easy bruising or bleeding. - Trouble breathing....

2. Score: 0.6686
   Signs and symptoms may include pain or swelling, fever, abscess, cough, high fever, headache, trouble breathing, and more. Although healthy people can also experience signs and symptoms of the disease...

3. Score: 0.6662
   - Breathing problems. - Cough. -  Fever....

🔍 Testing query: 'What symptoms suggest meningitis in a child?'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📋 Top 3 results:

1. Score: 0.7304
   Symptoms of meningitis, which may appear suddenly, often include high fever, severe and persistent headache, stiff neck, nausea, sensitivity to bright light, and vomiting. Changes in behavior such as ...

2. Score: 0.6691
   Changes in behavior such as confusion, sleepiness, and difficulty waking up may also occur. In infants, symptoms of meningitis or encephalitis may include fever, vomiting, lethargy, body stiffness, un...

3. Score: 0.6201
   The condition usually develops approximately 14-28 days after infection with group A Streptococcus bacteria, such as strep throat or scarlet fever. Early signs and symptoms may include sore throat; sw...

🔍 Testing query: 'When should I see a doctor for a headache?'


...


📋 Top 3 results:

1. Score: 0.7130
   Let your health care provider know if you have sudden, severe headaches. Get medical help right away if you have a headache after a blow to your head, or if you have a headache along with a stiff neck...

2. Score: 0.7099
   Not all headaches require a doctor's attention. But sometimes headaches warn of a more serious disorder. Let your health care provider know if you have sudden, severe headaches....

3. Score: 0.6899
   But sometimes headaches warn of a more serious disorder. Let your health care provider know if you have sudden, severe headaches. Get medical help right away if you have a headache after a blow to you...

🔍 Testing query: 'What are red flag symptoms for chest pain?'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📋 Top 3 results:

1. Score: 0.6805
   If you dont know whether your chest pain is angina or a heart attack, call 911. All chest pain should be checked by a doctor. Other common signs and symptoms of a heart attack include:
               ...

2. Score: 0.6769
   Medical History
                
Your doctor may ask whether you:
                
Have had a recent respiratory infection or flu-like illness
                
Have had a recent heart attack or injury...

3. Score: 0.6755
   - Trouble breathing. - Weakness or feeling tired. - Pain below the ribs....

🔍 Testing query: 'What diagnostic tests are recommended for persistent cough?'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📋 Top 3 results:

1. Score: 0.7168
   Other Tests
                
Your doctor may recommend other tests if you're in the hospital, have serious symptoms, are older, or have other health problems. Sputum test. Your doctor may look at a sa...

2. Score: 0.6903
   Physical Exam
                
To check for signs of problems related to cough, your doctor will use a stethoscope to listen to your lungs. He or she will listen for wheezing (a whistling or squeaky s...

3. Score: 0.6763
   - A cough that doesnt go away or gets worse over time. - Trouble breathing. - Wheezing....
